In [6]:
import process_observed_discharge as q
import os

import warnings
warnings.simplefilter("ignore")

### Unzip files and copy to common folder
First unzip the files in individual folders and then transfer the tmeseries files into a common folder. For this, use the uzip_files and copy_to_common_folder  functions in the process_observed_discharge module

In [ ]:
src_zipped_wallonie = 'D:/VUB/_data/hydrometrie_wallonie/rawfiles'
common_folder_wallonie = 'D:/VUB/_data/hydrometrie_wallonie/unzipped_all'

src_zipped_waterinfo = 'D:/VUB/_data/waterinfo/zipped'
common_folder_waterinfo = 'D:/VUB/_data/waterinfo/unzipped'

#For stations in Flanders, the station metadata is downloaded from the Waterinfo website using the py_waterinfo package.
#I decided to download the timeseries one by one because I struggled with the py_waterinfo package. It also helped me to skip data from canals
#The data has been processed using the download_waterinfo_data.ipynb notebook and saved as a csv file (waterinfo_Q_Stations.csv).
#The csv file contains the station name and coordinates.
#Since we do not need to use Q data from canals, we filter out the stations that are not rivers.

In [ ]:
#unzip the files (both Wallonie and Waterinfo)
q.unzip_files(src_zipped_waterinfo, common_folder_waterinfo)

In [4]:
#copy files to common folder
#q.copy_to_common_folder(src_zipped, common_folder)

In [ ]:
#Extract station timeseries and coordinates for Wallonie data
Q_dict, station_coords = q.extract_timeseries_wallonie(common_folder_wallonie)

#### Intersect waterinfo timeseries with station metadata
Since we do not need to use Q data from canals, we filter out the stations that are not rivers.